In [1]:
instrument = 'NIFTY'

In [2]:
import datetime
import numpy as np
import pandas as pd

In [3]:
from glob import glob

In [4]:
from IPython.display import clear_output

In [5]:
start_date = datetime.date(2023,1,1)
stop_date = datetime.date(2023,6,30)

In [6]:
files_path = f'../ML Data/fno/{instrument.lower()}'

In [7]:
all_files = glob(files_path+"/*")
all_files_df = pd.DataFrame(all_files,columns=['path'])
all_files_df['date'] = all_files_df['path'].apply(lambda x: x.split('/')[-1].split('.')[0])
all_files_df.sort_values('date',inplace=True)
all_files_df.reset_index(drop=True,inplace=True)

all_files_df['date'] = pd.to_datetime(all_files_df['date']).apply(lambda x: x.date())
all_files_df = all_files_df[(all_files_df['date'] >= start_date) & (all_files_df['date'] <= stop_date)]
all_files_df = all_files_df.set_index('date')

In [8]:
# take the expiries for managing the positions
# expiry_dates = {}

for date_idx, row in all_files_df.iterrows():

    print(date_idx)
    
    data = pd.DataFrame()
    data = pd.read_parquet(row['path'])

    expiry_date = None
    expiry_date = data[data['expiry_date'] >= date_idx]['expiry_date'].min()
    
    # expiry_dates[date_idx] = expiry_date

    all_files_df.loc[date_idx,'expiry_date'] = expiry_date
    
    clear_output(wait=True)

2023-06-30


In [9]:
all_files_df

,path,expiry_date
date,,
2023-01-02,../ML Data/fno/nifty/2023-01-02.parquet,2023-01-05
2023-01-03,../ML Data/fno/nifty/2023-01-03.parquet,2023-01-05
2023-01-04,../ML Data/fno/nifty/2023-01-04.parquet,2023-01-05
2023-01-05,../ML Data/fno/nifty/2023-01-05.parquet,2023-01-05
2023-01-06,../ML Data/fno/nifty/2023-01-06.parquet,2023-01-12
...,...,...
2023-06-23,../ML Data/fno/nifty/2023-06-23.parquet,2023-06-29
2023-06-26,../ML Data/fno/nifty/2023-06-26.parquet,2023-06-29
2023-06-27,../ML Data/fno/nifty/2023-06-27.parquet,2023-06-29


In [10]:
spot_data_filepath = f'../ML Data/spot/{instrument.lower()}.csv'

In [11]:
sma_f = 5
sma_s = 13

In [12]:
spot_data = pd.read_csv(spot_data_filepath)
spot_data['datetime'] = pd.to_datetime(spot_data['datetime'])
spot_data['date'] = spot_data['datetime'].dt.date
spot_data['time'] = spot_data['datetime'].dt.time

spot_data = spot_data[(spot_data['date'] >= start_date) & (spot_data['date'] <= stop_date)].reset_index(drop=True)
spot_data = spot_data.sort_values('datetime')

spot_data.set_index('datetime',inplace=True)

spot_data_1D = spot_data.resample('1D').apply({'open':'first','high':'max','low':'min','close':'last'})
spot_data_1D = spot_data_1D.dropna()
spot_data_1D['sma_f'] = spot_data_1D['close'].rolling(window=sma_f).mean()
spot_data_1D['sma_s'] = spot_data_1D['close'].rolling(window=sma_s).mean()

# lookahead removal
# since we cannot see history before it happens
spot_data_1D[[ 'sma_f', 'sma_s']] = spot_data_1D[[ 'sma_f', 'sma_s']].shift()

In [13]:
# generate signals
for datetime_idx, d_today in spot_data_1D[1:].iterrows():

    if datetime_idx.date() not in all_files_df.index:
        continue

    if (
        (d_today['sma_f'] > d_today['sma_s'])
    ):
        print(datetime_idx.date(),'LONG')
        all_files_df.loc[datetime_idx.date(),'direction'] = 'LONG'

    elif (
        (d_today['sma_f'] < d_today['sma_s'])
    ):
        print(datetime_idx.date(),'SHORT')
        all_files_df.loc[datetime_idx.date(),'direction'] = 'SHORT'

    else:
        print(datetime_idx.date(),'NEUTRAL')
        all_files_df.loc[datetime_idx.date(),'direction'] = 'NEUTRAL'
    
    clear_output(wait=True)

2023-05-18 LONG


In [14]:
all_files_df

,path,expiry_date,direction
date,,,
2023-01-02,../ML Data/fno/nifty/2023-01-02.parquet,2023-01-05,NaN
2023-01-03,../ML Data/fno/nifty/2023-01-03.parquet,2023-01-05,NEUTRAL
2023-01-04,../ML Data/fno/nifty/2023-01-04.parquet,2023-01-05,NEUTRAL
2023-01-05,../ML Data/fno/nifty/2023-01-05.parquet,2023-01-05,NEUTRAL
2023-01-06,../ML Data/fno/nifty/2023-01-06.parquet,2023-01-12,NEUTRAL
...,...,...,...
2023-06-23,../ML Data/fno/nifty/2023-06-23.parquet,2023-06-29,NaN
2023-06-26,../ML Data/fno/nifty/2023-06-26.parquet,2023-06-29,NaN
2023-06-27,../ML Data/fno/nifty/2023-06-27.parquet,2023-06-29,NaN
